import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical

In [1]:
import pandas

In [2]:
df = pandas.read_csv("C:\\Users\\sante\\Desktop\\мой нлп - Лист1-2.csv", encoding="utf-8")

# тестовые данные
valid = df["title"].isin(['Интерстеллар', "Омерзительная восьмерка", "Тройной форсаж: Токийский дрифт"])
test = df[valid]
del test['title']

# удалили наши данные(оставили данные для обучения)
df = df.loc[df['title'] != "Интерстеллар"]
df = df.loc[df['title'] != "Омерзительная восьмерка"]
df = df.loc[df['title'] != "Тройной форсаж: Токийский дрифт"]
del df['title']

In [3]:
 df.head

<bound method NDFrame.head of                                                   text  label
0    Дон Блут – один из самых известных анимационны...     -1
1    Отрицательных рецензий на этот мультфильм очен...     -1
2    \nЧестно признаюсь: в детстве я если и смотрел...     -1
3    \nВпервые я познакомилась с творением студии '...     -1
4    \nЯ узнала про мультфильм 'Анастасия' на продл...     -1
..                                                 ...    ...
585  Питер Джексон уже не тот...  Пишу как человек,...     -1
586  Ждали-ждали - дождались  Около двух лет назад ...     -1
587  Три часа скукоты  В том, что «Хоббит: Нежданно...     -1
588  что-то случилось с любимым нами Средиземьем  В...     -1
589  Детям фильм смотреть нельзя  Фильм не понравил...     -1

[590 rows x 2 columns]>

In [4]:
def tokenizeWord(word):
    word = tokenizer.tokenize(word)
    if len(word) == 1:
        word = word[0]
    else:
        word = ""
    return word


In [5]:

import pymorphy2
from nltk.tokenize import RegexpTokenizer
morph = pymorphy2.MorphAnalyzer()
tokenizer = RegexpTokenizer(r'\w+')
uniqWords = []
tupleIndex = 0
sentences = []
for tuple in df.values:
    tupleIndex+=1
    words = tuple[0].split()
    wordsList = []
    for word in words:
        wordNF = tokenizeWord(morph.parse(word)[0].normal_form)
        uniqWords.append(wordNF)
        wordsList.append(wordNF)
    sentences.append(wordsList)
tupleIndex = 0
testSentences = []
for tuple in test.values:
    tupleIndex+=1
    words = tuple[0].split()
    wordsList = []
    for word in words:
        wordNF = tokenizeWord(morph.parse(word)[0].normal_form)
        uniqWords.append(wordNF)
        wordsList.append(wordNF)
    testSentences.append(wordsList)

In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

print(len(uniqWords))

tokenizer = Tokenizer(num_words=len(uniqWords))
tokenizer.fit_on_texts(sentences)

vocab_size = len(tokenizer.word_index) + 1
maxlen = 500

X_train = tokenizer.texts_to_sequences(sentences)
X_test = tokenizer.texts_to_sequences(testSentences)

Using TensorFlow backend.


273477


In [7]:
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [8]:
print(vocab_size)


25916


In [9]:
from keras.utils import to_categorical
y_train = df["label"]
y_test = test["label"]

num_classes = 3

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)


In [10]:
import numpy as np
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath, encoding="utf-8") as f:
        for line in f:
            word, *vector = line.split()
            word = word.split("_")[0]
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [11]:
embedding_dim = 300
embedding_matrix = create_embedding_matrix('C:/Users/sante/Desktop/model.txt', tokenizer.word_index, embedding_dim)

In [12]:
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
print(embedding_matrix.shape)
print(nonzero_elements)
nonzero_elements / vocab_size

(25916, 300)
14052


0.5422133045223029

In [13]:
from keras.layers import Dense, Dropout, Activation, Input, LSTM
from keras.models import Sequential
from keras import layers
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=maxlen, 
                           trainable=False))
model.add(LSTM(300, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_classes, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 300)          7774800   
_________________________________________________________________
lstm_1 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 903       
Total params: 8,496,903
Trainable params: 722,103
Non-trainable params: 7,774,800
_________________________________________________________________


In [14]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [17]:

model.fit(X_train, y_train, epochs=30, batch_size=32)


Epoch 1/30
590/590 [==============================] - 36s 61ms/step - loss: 1.0675 - accuracy: 0.3407
Epoch 2/30
590/590 [==============================] - 36s 60ms/step - loss: 1.0634 - accuracy: 0.3492
Epoch 3/30
590/590 [==============================] - 36s 60ms/step - loss: 1.0610 - accuracy: 0.3458
Epoch 4/30
590/590 [==============================] - 36s 61ms/step - loss: 1.0552 - accuracy: 0.3475
Epoch 5/30
590/590 [==============================] - 36s 61ms/step - loss: 1.0327 - accuracy: 0.3763
Epoch 6/30
590/590 [==============================] - 36s 60ms/step - loss: 1.0040 - accuracy: 0.4424
Epoch 7/30
590/590 [==============================] - 36s 61ms/step - loss: 1.0114 - accuracy: 0.4695
Epoch 8/30
590/590 [==============================] - 36s 61ms/step - loss: 1.1088 - accuracy: 0.3983
Epoch 9/30
590/590 [==============================] - 36s 60ms/step - loss: 1.0452 - accuracy: 0.4322
Epoch 10/30
590/590 [==============================] - 36s 60ms/step - loss: 1.028

In [19]:
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test.argmax(axis=1), predictions.argmax(axis=1)))

              precision    recall  f1-score   support

           0       0.62      0.17      0.26        30
           1       0.39      0.73      0.51        30
           2       0.38      0.33      0.36        30

    accuracy                           0.41        90
   macro avg       0.47      0.41      0.38        90
weighted avg       0.47      0.41      0.38        90

